Importing data

In [28]:
# uncomment if using this code in RStudio
# setwd(dirname(rstudioapi::getActiveDocumentContext()$path))

library(lme4)
library(plyr)
library(xtable)
library(LMERConvenienceFunctions)
library(car)
# library(lmerTest) see also https://link.springer.com/article/10.3758/s13428-016-0809-y 

source("r_utils/mer-utils.R")
source("r_utils/regression-utils.R")

data_path = "C:/Users/Arkady/Google Drive/data/eyehandrdk/%s/processed/choices.txt"
output_table_path = "C:/Users/Arkady/Dropbox/Research/Journal papers/2018 CoM/tables/"

choice.data_1 = read.table(sprintf(data_path, "exp_1"), header=T)
choice.data_2 = read.table(sprintf(data_path, "exp_2"), header=T)

A bit of preprocessing to create variables for analyzing sequential effects

In [29]:
preprocess_data <- function(choice.data) {
    # any exclusions?
    stats.df = choice.data
    stats.df$subj_id = as.factor(stats.df$subj_id)

    # generate trial number
    max.trial = max(stats.df$trial_no)
    max.block = max(stats.df$block_n)
    stats.df$all_trial_no = (stats.df$session_no-1) * (max.block * max.trial) +
      (stats.df$block_no-1) * max.trial +
      stats.df$trial_no

    ## checking - can be commmented when not needed
    check.diffs = diff(stats.df$all_trial_no)
#     hist(check.diffs)
    check.diffs[check.diffs<0] # 3 resets
    check.diffs[check.diffs>1] # 36 diffs of +2 

    check = stats.df[c(check.diffs>1, T),] # check missing trials
    aggregate(all_trial_no~subj_id, check, length) # - poss cleaning?

    rm(check) # no need 
    ## checking complete

    # generate trial_unique
    stats.df$trial_unique = paste(stats.df$subj_id, stats.df$all_trial_no, sep = "_")

    # generate lasttrial.resp and lasttrial.dir
    lasttrial.details  = ddply(stats.df,
                               .(subj_id, session_no, block_no), summarize,
                               trial_no = trial_no,
                               response = response,
                               lasttrial.resp  = c(F, diff(response)==0),
                               direction = direction,
                               lasttrial.dir = c(F, diff(direction)==0),
                               trial_unique = trial_unique
    )

    # add lasttrial.resp and lasttrial.dir to stats.df
    stats.df = merge(stats.df, lasttrial.details[,
                        c("trial_unique", "lasttrial.resp", "lasttrial.dir")],
                        by = "trial_unique")

    # set up contrasts for lasttrial.resp and lasttrial.dir 
    contrasts(stats.df$lasttrial.resp) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$lasttrial.resp) # true is pos

    contrasts(stats.df$lasttrial.dir) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$lasttrial.dir) # true is pos

    contrasts(stats.df$is_com) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$is_com)# true is pos

    str(stats.df)

    return(stats.df)
}

stats.df_1 = preprocess_data(choice.data_1)
stats.df_2 = preprocess_data(choice.data_2)

'data.frame':	9564 obs. of  50 variables:
 $ trial_unique              : chr  "269_1" "269_10" "269_100" "269_1000" ...
 $ subj_id                   : Factor w/ 4 levels "269","275","391",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ session_no                : int  1 1 1 2 2 2 2 2 2 2 ...
 $ block_no                  : int  1 1 2 7 7 7 7 7 7 7 ...
 $ trial_no                  : int  1 10 40 40 41 42 43 44 45 46 ...
 $ is_practice               : Factor w/ 2 levels "False","True": 2 2 2 2 2 2 2 2 2 2 ...
 $ direction                 : num  0 180 180 180 0 180 0 180 0 0 ...
 $ coherence                 : num  0.512 0.128 0.064 0 0.032 0.064 0.256 0.256 0.032 0.032 ...
 $ duration                  : int  800 800 800 800 800 800 800 800 800 800 ...
 $ response                  : int  0 180 0 180 0 180 0 180 0 0 ...
 $ response_time             : num  3.097 2.504 1.147 0.935 1.208 ...
 $ is_correct                : Factor w/ 2 levels "False","True": 2 2 1 2 2 2 2 2 2 2 ...
 $ xflips                    : 

# Analysis 1: Probability of CoM as a function of coherence

In [39]:
run_analysis_1 <- function(stats.df, exp_name){
    rnd1.lmer = glmer(is_com ~ (1|subj_id), stats.df, family = binomial)

    rnd2.lmer = glmer(is_com ~ (c.(coherence)|subj_id), stats.df, family = binomial)

    rnd3.lmer = glmer(is_com ~ (poly(c.(coherence), 2, raw = T)|subj_id), stats.df, family = binomial)

    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer)
    rnd.anova
    # For both exp 1 and exp 2, rnd2 looks better than others

    choice.mer =  update(rnd2.lmer, .~. + c.(coherence))
    print(summary(choice.mer))

#     choice.mer =  update(rnd.lmer, .~. + poly(c.(coherence), 2, raw = T))
#     summary(choice.mer)

    # confidence intervals if required
    CIs = confint(choice.mer, parm="beta_",method="Wald")
    CIs.tab = cbind(est.OR=fixef(choice.mer),CIs)
    Odds.tab = exp(CIs.tab)
    print(Odds.tab,digits=3)

    choice.output = summary(choice.mer)$coefficients

    file_name = sprintf(paste(output_table_path, "is_com_vs_coh_%s.tex", sep=""), exp_name)
    print(xtable(choice.output, digits = c(2,2,2,2,-2), 
                 label = "tab:is_com_vs_coh",
                 caption = 'Probability of change-of-mind as a function of coherence'), 
          math.style.exponents = TRUE, type = "latex", file = file_name)
}

In [40]:
run_analysis_1(stats.df_1, "exp_1")

singular fit
singular fit
singular fit


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_com ~ (c.(coherence) | subj_id) + c.(coherence)
   Data: stats.df

     AIC      BIC   logLik deviance df.resid 
  2246.6   2282.4  -1118.3   2236.6     9559 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.3338 -0.2312 -0.1161 -0.0691 26.7033 

Random effects:
 Groups  Name          Variance Std.Dev. Corr
 subj_id (Intercept)   1.5172   1.2317       
         c.(coherence) 0.1619   0.4023   1.00
Number of obs: 9564, groups:  subj_id, 4

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -4.2752     0.6312  -6.773 1.26e-11 ***
c.(coherence)  -4.5830     1.0167  -4.508 6.56e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
c.(cohernc) 0.336 
convergence code: 0
singular fit

              est.OR   2.5 % 97.5 %
(Intercept)   0.0139 

In [41]:
run_analysis_1(stats.df_2, "exp_2")

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_com ~ (c.(coherence) | subj_id) + c.(coherence)
   Data: stats.df

     AIC      BIC   logLik deviance df.resid 
  1908.1   1942.5   -949.1   1898.1     7185 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.2180 -0.1976 -0.1800 -0.1559  8.3069 

Random effects:
 Groups  Name          Variance Std.Dev. Corr
 subj_id (Intercept)   0.04556  0.2135       
         c.(coherence) 3.08644  1.7568   0.84
Number of obs: 7190, groups:  subj_id, 3

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.5973     0.1476 -24.373   <2e-16 ***
c.(coherence)  -2.5216     1.1574  -2.179   0.0294 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
c.(cohernc) 0.740 
              est.OR   2.5 % 97.5 %
(Intercept)   0.0274 0.02052 0.0366
c.(coherence) 0.080

In both experiments, probability of CoM decreases with coherence

# Analysis 2

In [42]:
run_analysis_2 <- function(stats.df, exp_name){
    rnd.lmer = glmer(is_correct ~ (1|subj_id), 
                 stats.df[stats.df$coherence!=0,], # edit this line to include zero coh
                 family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff learning effect for each p
    rnd2.lmer = glmer(is_correct ~ (c.(l.(all_trial_no))|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of coherence
    # diff avg acc, diff coherence effect for each p
    rnd3.lmer = glmer(is_correct ~ (c.(coherence)|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff linear and quad learning effect for each p
    rnd3.quad.lmer = glmer(is_correct ~ (poly(coherence, 2, raw = T)|subj_id), 
                           stats.df[stats.df$coherence!=0,], 
                           family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff trial-to-trial stimulus effect for each p
    rnd4.lmer = glmer(is_correct ~ (lasttrial.dir|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff trial-to-trial stimulus effect and diff learning effect for each p
    rnd5.lmer = glmer(is_correct ~ (c.(l.(all_trial_no)) + c.(coherence)|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd.lmer with com and coherence
    rnd6.lmer = glmer(is_correct ~ (is_com + c.(coherence)|subj_id),
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)

    # doesn't converge
    # rnd5.quad.lmer = glmer(is_correct ~ (c.(l.(all_trial_no)) + c.(coherence) + c.(I(coherence^2))|subj_id), stats.df, family = binomial)
    # rnd6.lmer = glmer(is_correct ~ (c.(l.(all_trial_no)) + lasttrial.dir + c.(coherence)|subj_id), stats.df, family = binomial)
    #rnd7.lmer = glmer(is_correct ~ (c.(l.(all_trial_no)) + lasttrial.dir + lasttrial.resp + c.(coherence)|subj_id), stats.df, family = binomial)

    rnd.anova = anova(rnd.lmer, rnd2.lmer, rnd3.lmer, rnd3.quad.lmer, rnd4.lmer, rnd5.lmer, rnd6.lmer)#, rnd7.lmer)
    rnd.anova
}

singular fit
singular fit
singular fit
singular fit


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
rnd.lmer,2,9049.675,9063.643,-4522.838,9045.675,NA,NA,NA
rnd2.lmer,4,9053.504,9081.439,-4522.752,9045.504,0.1717519,2,9.177081e-01
rnd3.lmer,4,7603.024,7630.959,-3797.512,7595.024,1450.4796178,0,0.000000e+00
rnd4.lmer,4,8940.685,8968.620,-4466.343,8932.685,0.0000000,0,1.000000e+00
rnd3.quad.lmer,7,7607.903,7656.790,-3796.951,7593.903,1338.7821620,3,5.659347e-290
rnd5.lmer,7,7606.428,7655.315,-3796.214,7592.428,1.4747059,0,0.000000e+00
rnd6.lmer,7,7596.597,7645.484,-3791.298,7582.597,9.8313284,0,0.000000e+00


In [ ]:
run_analysis_2(stats.df_1, "exp_1")

In [43]:
run_analysis_2(stats.df_2, "exp_2")

singular fit
singular fit
singular fit
singular fit


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
rnd.lmer,2,6659.350,6672.747,-3327.675,6655.350,NA,NA,NA
rnd2.lmer,4,6642.783,6669.578,-3317.392,6634.783,20.566384,2,3.420317e-05
rnd3.lmer,4,5714.391,5741.186,-2853.196,5706.391,928.392027,0,0.000000e+00
rnd4.lmer,4,6647.144,6673.938,-3319.572,6639.144,0.000000,0,1.000000e+00
rnd3.quad.lmer,7,5717.951,5764.841,-2851.976,5703.951,935.192570,3,2.057633e-202
rnd5.lmer,7,5698.088,5744.977,-2842.044,5684.088,19.863316,0,0.000000e+00
rnd6.lmer,7,5691.882,5738.772,-2838.941,5677.882,6.205285,0,0.000000e+00


In both experiments, the simple coherence per subject random effect might suffice